In [32]:
import pandas as pd 
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn import metrics


In [4]:
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')

In [5]:
data_preprocessed.head(5)

,reason_1,reason_2,reason_3,reason_4,Month_Value,Weekday_Value,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [6]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [7]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)
data_preprocessed['Excessive Absenteism'] = targets

In [8]:
data_preprocessed

,reason_1,reason_2,reason_3,reason_4,Month_Value,Weekday_Value,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,5,2,179,22,40,237.656,22,1,2,0,8,1
696,1,0,0,0,5,2,225,26,28,237.656,24,0,1,2,3,0
697,1,0,0,0,5,3,330,16,28,237.656,25,1,0,0,8,1
698,0,0,0,1,5,3,235,16,32,237.656,25,1,0,0,2,0


In [9]:
targets.sum() /  targets.shape[0]

0.45571428571428574

In [10]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours'], axis=1)

### inputs

In [11]:
data_with_targets.shape

(700, 15)

In [12]:
unscaled_inputs = data_with_targets.iloc[:, 0:-1]

In [17]:
absenteism_scaler = StandardScaler()
absenteism_scaler.fit(unscaled_inputs)
scaled_inputs = absenteism_scaler.transform(unscaled_inputs)

StandardScaler()

In [21]:
scaled_inputs.shape

(700, 14)

### Split data and shuffle

In [30]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size=0.8, random_state=20)

In [31]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(560, 14) (140, 14) (560,) (140,)


In [33]:
reg = LogisticRegression()

In [34]:
reg.fit(x_train, y_train)

LogisticRegression()

In [36]:
#accuracy
reg.score(x_train, y_train)

0.7839285714285714

### manually check accuracy

In [52]:
model_output = reg.predict(x_train)

np.sum(model_output == y_train) / (model_output == y_train).shape[0]


0.7839285714285714